In [1]:
# For Mounting Google Drive when using Colab
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

%cd gdrive/MyDrive/h23q-data

! ls

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
normal_files = [f'{i}-nginx-ben.csv' for i in range(1, 5) if i!=6]
abnormal_files = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('-nginx-mal.csv')]

In [ ]:
# REMOVE NON NUMERICAL COLUMNS
df = pd.read_csv(normal_files[0])
columns_list = df.columns.tolist()
cols_to_keep = ['src_port', 'dst_port', 'protocol', 'flow_duration', 'flow_byts_s', 'flow_pkts_s', 'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts', 'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max', 'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std', 'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean', 'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean', 'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len', 'fwd_seg_size_min', 'fwd_act_data_pkts', 'flow_iat_mean', 'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot', 'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std', 'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean', 'bwd_iat_std', 'down_up_ratio', 'pkt_size_avg', 'init_fwd_win_byts', 'init_bwd_win_byts', 'active_max', 'active_min', 'active_mean', 'active_std', 'idle_max', 'idle_min', 'idle_mean', 'idle_std', 'fwd_byts_b_avg', 'fwd_pkts_b_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg', 'fwd_blk_rate_avg', 'bwd_blk_rate_avg', 'fwd_seg_size_avg', 'bwd_seg_size_avg', 'cwe_flag_count', 'subflow_fwd_pkts', 'subflow_bwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_byts']
columns_to_drop_1 = [c for c in columns_list if c not in cols_to_keep]

In [ ]:
# ======== FEATURE SELECTION ==============
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif

# Define file paths for class 0 and class 1
class0_files = normal_files
class1_files = abnormal_files

# Load data for class 0
class0_data = pd.concat([pd.read_csv(file) for file in class0_files], axis=0)

# Load data for class 1
class1_data = pd.concat([pd.read_csv(file) for file in class1_files], axis=0)

# Combine data from both classes
data = pd.concat([class0_data, class1_data], axis=0)

# Drop specified columns
data.drop(columns=columns_to_drop_1, inplace=True, errors='ignore')

# Create target variable indicating class labels
target = []
target.extend([0]* len(class0_data))
for i in range(len(class1_files)):
  file=class1_files[i]
  tempdata = pd.read_csv(file)
  target.extend([i]*len(tempdata))

In [ ]:
print(len(data))
print(len(target))

8135
8135


In [ ]:
# Perform ANOVA feature selection
selector = SelectKBest(score_func=f_classif, k=32)  # Select top 5 features, adjust k as needed
selected_features = selector.fit_transform(data, target)

# Get indices of selected features
selected_indices = selector.get_support(indices=True)

# Get names of selected features
selected_feature_names = data.columns[selected_indices]

print("Selected features:")
print(selected_feature_names)

Selected features:
Index(['src_port', 'protocol', 'flow_duration', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_std', 'bwd_pkt_len_max',
       'bwd_pkt_len_min', 'bwd_pkt_len_mean', 'bwd_pkt_len_std', 'pkt_len_max',
       'pkt_len_min', 'pkt_len_mean', 'pkt_len_std', 'pkt_len_var',
       'fwd_header_len', 'fwd_seg_size_min', 'flow_iat_max', 'fwd_iat_tot',
       'fwd_iat_max', 'down_up_ratio', 'pkt_size_avg', 'init_fwd_win_byts',
       'active_min', 'idle_max', 'idle_mean', 'idle_std', 'fwd_pkts_b_avg',
       'bwd_byts_b_avg', 'bwd_pkts_b_avg', 'bwd_blk_rate_avg',
       'bwd_seg_size_avg'],
      dtype='object')


In [ ]:
selected_feature_names=['src_port', 'protocol', 'flow_duration', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_std', 'bwd_pkt_len_max',
       'bwd_pkt_len_min', 'bwd_pkt_len_mean', 'bwd_pkt_len_std',
       'fwd_header_len', 'bwd_header_len', 'flow_iat_max',
       'fwd_iat_tot', 'fwd_iat_max', 'down_up_ratio',
        'idle_max', 'idle_mean', 'idle_std',
       'fwd_pkts_b_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'bwd_blk_rate_avg']